In [ ]:
!pip install pyspark

In [ ]:
# Census-Income (KDD) Data Set
# https://archive.ics.uci.edu/ml/machine-learning-databases/census-income-mld/

In [ ]:
import numpy as np
from pyspark.sql import SparkSession
import pandas as pd

In [ ]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
dataframe = pd.read_csv('/content/census-income.data')
dataframe.head()

,73,Not in universe,0,0.1,High school graduate,0.2,Not in universe.1,Widowed,Not in universe or children,Not in universe.2,White,All other,Female,Not in universe.3,Not in universe.4,Not in labor force,0.3,0.4,0.5,Nonfiler,Not in universe.5,Not in universe.6,Other Rel 18+ ever marr not in subfamily,Other relative of householder,1700.09,?,?.1,?.2,Not in universe under 1 year old,?.3,0.6,Not in universe.7,United-States,United-States.1,United-States.2,Native- Born in the United States,0.7,Not in universe.8,2,0.8,95,- 50000.
0,58,Self-employed-not incorporated,4,34,Some college but no degree,0,Not in universe,Divorced,Construction,Precision production craft & repair,White,All other,Male,Not in universe,Not in universe,Children or Armed Forces,0,0,0,Head of household,South,Arkansas,Householder,Householder,1053.55,MSA to MSA,Same county,Same county,No,Yes,1,Not in universe,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,52,94,- 50000.
1,18,Not in universe,0,0,10th grade,0,High school,Never married,Not in universe or children,Not in universe,Asian or Pacific Islander,All other,Female,Not in universe,Not in universe,Not in labor force,0,0,0,Nonfiler,Not in universe,Not in universe,Child 18+ never marr Not in a subfamily,Child 18 or older,991.95,?,?,?,Not in universe under 1 year old,?,0,Not in universe,Vietnam,Vietnam,Vietnam,Foreign born- Not a citizen of U S,0,Not in universe,2,0,95,- 50000.
2,9,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Children or Armed Forces,0,0,0,Nonfiler,Not in universe,Not in universe,Child <18 never marr not in subfamily,Child under 18 never married,1758.14,Nonmover,Nonmover,Nonmover,Yes,Not in universe,0,Both parents present,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0,94,- 50000.
3,10,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Children or Armed Forces,0,0,0,Nonfiler,Not in universe,Not in universe,Child <18 never marr not in subfamily,Child under 18 never married,1069.16,Nonmover,Nonmover,Nonmover,Yes,Not in universe,0,Both parents present,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0,94,- 50000.
4,48,Private,40,10,Some college but no degree,1200,Not in universe,Married-civilian spouse present,Entertainment,Professional specialty,Amer Indian Aleut or Eskimo,All other,Female,No,Not in universe,Full-time schedules,0,0,0,Joint both under 65,Not in universe,Not in universe,Spouse of householder,Spouse of householder,162.61,?,?,?,Not in universe under 1 year old,?,1,Not in universe,Philippines,United-States,United-States,Native- Born in the United States,2,Not in universe,2,52,95,- 50000.


In [ ]:
# # information from data table
# # 1. column names 
# print(list(dataframe.columns))
# col_names = list(dataframe.columns) 
# # 2. column data types
# size = len(col_names)
# for i in range(size):
#    print(col_names[i],dataframe.dtypes[i])

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
#Encode dataframe
LE = LabelEncoder()
dataframe[' Not in universe'] = LE.fit_transform(dataframe[' Not in universe'])
dataframe[' High school graduate'] = LE.fit_transform(dataframe[' High school graduate'])
dataframe[' Widowed'] = LE.fit_transform(dataframe[' Widowed'])
dataframe[' Not in universe or children'] = LE.fit_transform(dataframe[' Not in universe or children'])
dataframe[' Female'] = LE.fit_transform(dataframe[' Female'])
dataframe[' Not in labor force'] = LE.fit_transform(dataframe[' Not in labor force'])
dataframe[' Other Rel 18+ ever marr not in subfamily'] = LE.fit_transform(dataframe[' Other Rel 18+ ever marr not in subfamily'])
dataframe[' Other relative of householder'] = LE.fit_transform(dataframe[' Other relative of householder'])
dataframe[' United-States'] = LE.fit_transform(dataframe[' United-States'])
dataframe[' Native- Born in the United States'] = LE.fit_transform(dataframe[' Native- Born in the United States'])
print(dataframe[:])

        73   Not in universe   0   0.1  ...   2   0.8  95   - 50000.
0       58                 6   4    34  ...   2    52  94   - 50000.
1       18                 3   0     0  ...   2     0  95   - 50000.
2        9                 3   0     0  ...   0     0  94   - 50000.
3       10                 3   0     0  ...   0     0  94   - 50000.
4       48                 4  40    10  ...   2    52  95   - 50000.
...     ..               ...  ..   ...  ...  ..   ...  ..        ...
199517  87                 3   0     0  ...   2     0  95   - 50000.
199518  65                 5  37     2  ...   2    52  94   - 50000.
199519  47                 3   0     0  ...   2    52  95   - 50000.
199520  16                 3   0     0  ...   2     0  95   - 50000.
199521  32                 4  42    30  ...   2    52  94   - 50000.

[199522 rows x 42 columns]


In [ ]:
dataframe = dataframe.apply(LabelEncoder().fit_transform)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
#Scale dataframe
dataframe = MinMaxScaler().fit_transform(dataframe)
dataframe

array([[0.64444444, 0.75      , 0.07843137, ..., 1.        , 0.        ,
        0.        ],
       [0.2       , 0.375     , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.1       , 0.375     , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.52222222, 0.375     , 0.        , ..., 1.        , 1.        ,
        0.        ],
       [0.17777778, 0.375     , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.35555556, 0.5       , 0.82352941, ..., 1.        , 0.        ,
        0.        ]])

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Split and RDD data
split_data, test_data = train_test_split(dataframe, test_size=100)
data_rdd_train = spark.sparkContext.parallelize(split_data)
data_rdd_train.take(5)

[array([0.48888889, 0.5       , 0.84313725, 0.2173913 , 0.875     ,
        0.        , 1.        , 0.33333333, 0.2173913 , 0.71428571,
        1.        , 0.        , 0.        , 0.5       , 0.6       ,
        0.14285714, 0.        , 0.        , 0.28842248, 0.4       ,
        0.6       , 0.72      , 1.        , 1.        , 0.70640988,
        0.        , 0.        , 0.        , 0.5       , 0.        ,
        0.33333333, 1.        , 0.76190476, 0.95238095, 0.95238095,
        1.        , 0.        , 0.5       , 1.        , 0.75      ,
        1.        , 0.        ]),
 array([0.94444444, 0.375     , 0.        , 0.        , 0.3125    ,
        0.        , 1.        , 0.66666667, 0.60869565, 0.42857143,
        1.        , 0.        , 0.        , 0.5       , 0.6       ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.6       , 0.72      , 0.7027027 , 0.85714286, 0.64955561,
        0.77777778, 0.75      , 0.77777778, 1.        , 0.66666667,
        0.    

In [ ]:
import math

In [ ]:
# function to get distane
def get_distance(point1, point2):
  distance = np.sum((point1 - point2)**2)
  return distance

In [ ]:
# func KNN
def calcKNN(rdd_train, p):
  rdd = data_rdd_train.map(lambda x: (x[-1], get_distance(x[:-1], p[:-1])))
  return rdd.sortBy(lambda x: x[1], ascending=True).take(1)[0][0]

In [ ]:
import time

In [ ]:
# data_rdd_test = spark.sparkContext.parallelize(test_data)
# start = time.time()
# result = data_rdd_test.map(lambda x: calcKNN(data_rdd_train, x))
# print(time.time() - start)

0.0003142356872558594


In [ ]:
# result.collect()

In [ ]:
# Calc KNN
start = time.time()
result = []
for i in test_data:
  result.append(calcKNN(data_rdd_train, i))
print(time.time() - start)

879.9534842967987


In [ ]:
# Count Accurancy
temp = 0
for i in range(len(test_data)):
  if result[i] == np.array(test_data[i][-1], dtype=np.float64):
    temp = temp + 1
print('Accurancy: ', temp/len(test_data))

Accurancy:  0.96


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(result, test_data[:,-1]))

              precision    recall  f1-score   support

         0.0       0.97      0.99      0.98        96
         1.0       0.50      0.25      0.33         4

    accuracy                           0.96       100
   macro avg       0.73      0.62      0.66       100
weighted avg       0.95      0.96      0.95       100





---

